In [1]:
import os
from typing import Literal

from openai import OpenAI
from openai.types.chat import ChatCompletionUserMessageParam
from pydantic import BaseModel, create_model

In [2]:
def construct_type(labels: dict[str, list[str]]) -> type[BaseModel]:
    fields = {key: (Literal[*values], ...) for key, values in labels.items()}  # ty:ignore[invalid-type-form]

    return create_model("DynamicModel", **fields)  # ty:ignore[no-matching-overload]

In [3]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
)

model = "google/gemma-3-27b-it:free"

In [4]:
completion = client.chat.completions.create(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "こんにちわ！",
        }
    ],
)
print(completion.choices[0].message.content)

はい、こんにちは！何かお手伝いできることはありますか？😊



In [5]:
template = {
    "color": ["red", "green", "blue"],
    "size": ["S", "M", "L", "XL"],
}

DynamicModel = construct_type(template)

prompt = (
    "Given the following sentence, classify the color and size mentioned:\n\n"
    "Sentence: '{sentence}'"
)

sentence = "I have a green shirt in size M."

messages: list[ChatCompletionUserMessageParam] = [
    {"role": "user", "content": prompt.format(sentence=sentence)}
]

response = client.chat.completions.parse(
    model=model,
    messages=messages,
    response_format=DynamicModel,
)

if parsed := response.choices[0].message.parsed:
    print(parsed.color)  # ty:ignore[unresolved-attribute]
    print(parsed.size)  # ty:ignore[unresolved-attribute]

green
M
